In [20]:
!pip install scipy

In [21]:
import numpy as np
from scipy.io import loadmat

In [17]:
# phi-product utils

def apply_phi_transform(tensor, UU):
    """Apply unitary transform UU along 3rd mode"""
    n1, n2, n3 = tensor.shape
    transformed = np.zeros_like(tensor)
    for i in range(n1):
        for j in range(n2):
            transformed[i, j, :] = UU @ tensor[i, j, :]
    return transformed

def apply_phi_inverse(tensor, UU):
    """Apply inverse unitary transform UU^T along 3rd mode"""
    n1, n2, n3 = tensor.shape
    inverse = np.zeros_like(tensor)
    for i in range(n1):
        for j in range(n2):
            inverse[i, j, :] = UU.T @ tensor[i, j, :]
    return inverse

def phi_product(A, B, UU):
    """
    Multiply two tensors A and B using the Phi-product.

    Parameters:
        A: Tensor of shape (n1, n2, n3)
        B: Tensor of shape (n2, n4, n3)
        UU: Unitary transform matrix of shape (n3, n3)

    Returns:
        C = A *phi B, of shape (n1, n4, n3)
    """
    assert A.shape[1] == B.shape[0], "Tensor inner dimensions must match."
    assert A.shape[2] == B.shape[2] == UU.shape[0], "Transform dimension mismatch."

    # Step 1: Transform both tensors along mode-3
    A_phi = apply_phi_transform(A, UU)
    B_phi = apply_phi_transform(B, UU)

    n1, n2, n3 = A.shape
    _, n4, _ = B.shape
    C_phi = np.zeros((n1, n4, n3))

    # Step 2: Multiply frontal slices in transformed domain
    for k in range(n3):
        C_phi[:, :, k] = A_phi[:, :, k] @ B_phi[:, :, k]

    # Step 3: Inverse transform result
    C = apply_phi_inverse(C_phi, UU)
    return C

def phi_svd(tensor, UU):
    """
    Perform transformed tensor SVD using general unitary transform UU.
    Returns U, S, V such that: tensor ≈ U *phi S *phi V^T
    """
    # Step 1: Apply transform Phi along mode-3
    tensor_phi = apply_phi_transform(tensor, UU)

    n1, n2, n3 = tensor.shape
    U_phi = np.zeros((n1, n1, n3))
    S_phi = np.zeros((n1, n2, n3))
    V_phi = np.zeros((n2, n2, n3))

    # Step 2: SVD on each frontal slice in transform domain
    for k in range(n3):
        U_k, S_k, Vh_k = np.linalg.svd(tensor_phi[:, :, k], full_matrices=False)
        U_phi[:, :, k] = U_k
        S_phi[:, :, k] = np.diag(S_k)
        V_phi[:, :, k] = Vh_k

    # Step 3: Inverse transform Phi to get U, S, V
    U = apply_phi_inverse(U_phi, UU)
    S = apply_phi_inverse(S_phi, UU)
    V = apply_phi_inverse(V_phi, UU)

    return U, S, V


In [18]:
# t-SVD (Data) code

def prox_l1(X, tau):
    """Proximal operator for L1 norm (soft thresholding)"""
    return np.sign(X) * np.maximum(np.abs(X) - tau, 0)

def prox_ttnn(Y, UU, tau):
    """
    Proximal operator for the Transformed Tubal Nuclear Norm (TTNN)
    using t-SVD in the Phi domain.

    Parameters:
        Y   : Input tensor (3D)
        UU  : Unitary transform matrix (e.g., DCT, learned)
        tau : Threshold parameter (float)

    Returns:
        Thresholded tensor (3D) after applying TTNN shrinkage
    """
    # Step 1: Compute t-SVD in Phi domain
    U, S, V = phi_svd(Y, UU)

    # Step 2: Soft-threshold the singular values
    n1, n2, n3 = S.shape
    for k in range(n3):
        s = np.diag(S[:, :, k])
        s_thresh = np.maximum(s - tau, 0)
        S[:, :, k] = np.diag(s_thresh)

    # Step 3: Reconstruct the tensor: U *phi S *phi V^T
    return phi_product(phi_product(U, S, UU), V, UU)

def tensor_complete_t_svd_data(UU, X, Omega, dim=None, tol=1e-4, beta=1e-2, MaxIte=500, mu=1e-2, gamma=1.6, Z0=None, L0=None, E0=None):

    Z0 = Z0 if Z0 else np.zeros_like(X)
    L0 = L0 if L0 else np.zeros_like(X)
    E0 = E0 if E0 else np.zeros_like(X)

    n1, n2, n3 = X.shape
    BarOmega = 1 - Omega
    aaa = []

    for k in range(MaxIte):
        # Update Mbar
        Mbar = L0 + E0 - Z0 / beta
        Mbar = X * Omega + Mbar * BarOmega

        # Update L using TTNN proximal operator
        L = prox_ttnn(UU, Mbar + Z0 / beta - E0, 1 / beta)

        # Update M
        M = L + E0 - Z0 / beta
        M = X * Omega + M * BarOmega

        # Update E using soft thresholding
        E = prox_l1(M + Z0 / beta - L, mu / beta)

        # Update Lagrange multiplier Z
        Z = Z0 - gamma * beta * (L + E - M)

        # Check convergence
        if k > 30:
            etax = L - prox_ttnn(UU, L + Z, 1)
            etax = np.linalg.norm(etax.ravel()) / (1 + np.linalg.norm(Z.ravel()) + np.linalg.norm(L.ravel()))

            etae = E - prox_l1(Z + E, mu)
            etae = np.linalg.norm(etae.ravel()) / (1 + np.linalg.norm(Z.ravel()) + np.linalg.norm(E.ravel()))

            etap = L + E - M
            etap = np.linalg.norm(etap.ravel()) / (1 + np.linalg.norm(L.ravel()) + np.linalg.norm(M.ravel()) + np.linalg.norm(E.ravel()))

            aaa = [etax, etae, etap]
            eta = max([etax, etae, etap])

            if eta <= tol:
                break

        L0, Z0, E0 = L, Z, E

    return L, E, aaa

In [19]:
# t-SVD (Fourier) code

def t_product(A, B):
    """
    Compute the t-product of two 3D tensors A and B using FFT.

    Parameters:
        A : np.ndarray of shape (n1, n2, n3)
        B : np.ndarray of shape (n2, n4, n3)

    Returns:
        C : np.ndarray of shape (n1, n4, n3), the t-product A * B
    """
    assert A.shape[1] == B.shape[0], "Inner tensor dimensions must match"
    assert A.shape[2] == B.shape[2], "3rd dimension (tubes) must match"

    n1, n2, n3 = A.shape
    _, n4, _ = B.shape

    # Step 1: FFT along the 3rd mode
    A_fft = np.fft.fft(A, axis=2)
    B_fft = np.fft.fft(B, axis=2)

    # Step 2: Slice-wise matrix multiplication
    C_fft = np.zeros((n1, n4, n3), dtype=complex)
    for i in range(n3):
        C_fft[:, :, i] = A_fft[:, :, i] @ B_fft[:, :, i]

    # Step 3: Inverse FFT along the 3rd mode
    C = np.fft.ifft(C_fft, axis=2)

    return C.real  # discard imaginary part (should be numerically negligible)

def t_svd_fft(tensor):
    """
    Compute the t-SVD of a 3D tensor using FFT along the 3rd dimension.

    Returns:
        U, S, V: tensors such that tensor ≈ U * S * V^T
    """
    n1, n2, n3 = tensor.shape

    # Step 1: Apply FFT along the 3rd mode
    tensor_fft = np.fft.fft(tensor, axis=2)

    # Step 2: Perform SVD on each frontal slice in the Fourier domain
    U_fft = np.zeros((n1, n1, n3), dtype=complex)
    S_fft = np.zeros((n1, n2, n3), dtype=complex)
    V_fft = np.zeros((n2, n2, n3), dtype=complex)

    for k in range(n3):
        U_k, S_k, Vh_k = np.linalg.svd(tensor_fft[:, :, k], full_matrices=False)
        U_fft[:, :, k] = U_k
        S_fft[:, :, k] = np.diag(S_k)
        V_fft[:, :, k] = Vh_k

    # Step 3: Apply inverse FFT to get U, S, V in original domain
    U = np.fft.ifft(U_fft, axis=2).real
    S = np.fft.ifft(S_fft, axis=2).real
    V = np.fft.ifft(V_fft, axis=2).real

    return U, S, V

def prox_tnn(Y, tau):
    """
    Proximal operator for the Tubal Nuclear Norm (TTNN)
    using t-SVD in the fourier domain.

    Parameters:
        Y   : Input tensor (3D)
        UU  : Unitary transform matrix (e.g., DCT, learned)
        tau : Threshold parameter (float)

    Returns:
        Thresholded tensor (3D) after applying TNN shrinkage
    """
    # Step 1: Compute t-SVD in Phi domain
    U, S, V = t_svd_fft(Y)

    # Step 2: Soft-threshold the singular values
    n1, n2, n3 = S.shape
    for k in range(n3):
        s = np.diag(S[:, :, k])
        s_thresh = np.maximum(s - tau, 0)
        S[:, :, k] = np.diag(s_thresh)

    # Step 3: Reconstruct the tensor: U *phi S *phi V^T
    return t_product(t_product(U, S), V)

def tensor_complete_t_svd_fourier(X, Omega, dim=None, tol=1e-4, beta=1e-2, MaxIte=500, mu=1e-2, gamma=1.6, Z0=None, L0=None, E0=None):

    Z0 = Z0 if Z0 else np.zeros_like(X)
    L0 = L0 if L0 else np.zeros_like(X)
    E0 = E0 if E0 else np.zeros_like(X)

    n1, n2, n3 = X.shape
    BarOmega = 1 - Omega
    aaa = []

    for k in range(MaxIte):
        # Update Mbar
        Mbar = L0 + E0 - Z0 / beta
        Mbar = X * Omega + Mbar * BarOmega

        # Update L using TTNN proximal operator
        L = prox_tnn(Mbar + Z0 / beta - E0, 1 / beta)

        # Update M
        M = L + E0 - Z0 / beta
        M = X * Omega + M * BarOmega

        # Update E using soft thresholding
        E = prox_l1(M + Z0 / beta - L, mu / beta)

        # Update Lagrange multiplier Z
        Z = Z0 - gamma * beta * (L + E - M)

        # Check convergence
        if k > 30:
            etax = L - prox_tnn(L + Z, 1)
            etax = np.linalg.norm(etax.ravel()) / (1 + np.linalg.norm(Z.ravel()) + np.linalg.norm(L.ravel()))

            etae = E - prox_l1(Z + E, mu)
            etae = np.linalg.norm(etae.ravel()) / (1 + np.linalg.norm(Z.ravel()) + np.linalg.norm(E.ravel()))

            etap = L + E - M
            etap = np.linalg.norm(etap.ravel()) / (1 + np.linalg.norm(L.ravel()) + np.linalg.norm(M.ravel()) + np.linalg.norm(E.ravel()))

            aaa = [etax, etae, etap]
            eta = max([etax, etae, etap])

            if eta <= tol:
                break

        L0, Z0, E0 = L, Z, E

    return L, E, aaa

In [28]:

# Load the .mat file
data = loadmat('JasperRidge2_R198.mat')

# List available variables
print(data.keys())

# # Access a specific variable
Y = data['Y']
print(len(Y), len(Y[0]))


dict_keys(['__header__', '__version__', '__globals__', 'Region', 'SlectBands', 'nRow', 'nCol', 'nBand', 'Y', 'maxValue'])
198 10000
